# Rotate GEM2.5 Research Forecast Winds

This notebook documents the process for correcting the wind direction
issue in the GEM2.5 research model forecast files
(/results/forcing/atmospheric/GEM2.5/research/).



Our collaborators at EC discovered in early March 2016 that:

> [...]the U and V components of the wind in the netcdfs are labelled as the zonal and 
> meridional components of the wind while, in fact, they are the U and V components of the wind 
> on the model grid. In other words, the U (V) component of the wind does [not] lies[sic] in 
> the East-West (North-South) direction but along the horizontal (vertical) axis of the model 
> grid.

Starting on 7-Apr-2016 the files we download from EC have correct zonal/meridional wind components.
The files that we downloaded prior to that date need to have a rotational correction algorithm applied.
The date range of the affected files is 10-Sep-2014 through 6-Apr-2016.

The rotational correction algorithm provided by EC is:

1. Read the U and V components of the wind on the model grid.
Let's call them `u_grid`  and `v_grid` which are both 2D arrays.

2. Convert to polar coordinates:
```python
modulus = sqrt(u_grid^2 + v_grid^2)
rad_to_deg = 180 / pi
angle_grid = arctan2(-u_grid, -v_grid) * rad_to_deg
```
where:
 * Angles here are defined according to the meteorological wind direction convention.
 * We use the 2 argument 4 quadrant arctangent function (the `ATAN(Y,X)` of IDL or the `atan2(Y,X)` of Matlab)

3. Read the `correction_angle` variable in the provided `corr_fact_west.nc` file
(stored as `/results/forcing/atmospheric/GEM2.5/research/corr_fact_west.nc`).
This is a 2D array of the same dimension as u_grid and v_grid.

4. Apply the correction angle using the following formula:
```python
angle_WESN = angle_grid + correction_angle
```

5. Calculate the W-E and S-N components of the wind:
```python
deg_to_rad = pi / 180
u_WE = -modulus * sin(angle_WESN * deg_to_rad)  # the zonal component of the wind
v_SN = -modulus * cos(angle_WESN * deg_to_rad)  # the meridional component of the wind
```

In [53]:
import arrow
import numpy as np
from pathlib import Path
import xarray as xr

In [57]:
corrections = '/results/forcing/atmospheric/GEM2.5/research/corr_fact_west.nc'
forecast = '/results/forcing/atmospheric/GEM2.5/research/res_y2016m04d06.nc'
corrected_forecast = str(Path(forecast).with_suffix('.corr.nc'))

with xr.open_dataset(corrections) as corr:
    with xr.open_dataset(forecast) as ds:
        modulus = np.sqrt(ds.u_wind**2 + ds.v_wind**2)
        angle_grid = np.rad2deg(np.arctan2(-ds.u_wind, -ds.v_wind))
        angle_WESN = angle_grid + corr.correction_angle
        u_WE = -modulus * np.sin(np.deg2rad(angle_WESN))
        v_SN = -modulus * np.cos(np.deg2rad(angle_WESN))
        ds.u_wind.values = u_WE
        ds.v_wind.values = v_SN
        ds.attrs['history'] = '\n'.join((
            ds.history,
            '{} Correct wind components to true zonal/meridional values '
            'using EC supplied angles and algorithm.'
            .format(arrow.now().format('YYYY-MM-DD HH:mm:ss'))
        ))
        ds.to_netcdf(corrected_forecast, 'w')